In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'indian-currency-note-images-dataset-2020:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F886101%2F1505106%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240402%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240402T043623Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Daac0e43532c543766e13fea2ad205a5495c2f23e693645b343d4bcab75f5fa21b19ed9f6269243209a296f8deb3862d4c15f693703a2d37f85b9feab273b3daac21c7de4ab5ca0e5ddb4e1c6a33e35f472035755108d58044d94bd20fe6e216615b72b7c011973dec3666ae55d08a3489c01c00a9c94114baaacbb8cf9bbd12223c6459a6b2f13f4ecb969ae68cb6bb8df62013fc2e3a22709dea634ae996ed1e9582bba4f9a51da39090f08d40923c823916fa84f67eafead0d3d5f5e8a47078885234baf0fe2cc5551e25e23a838a1376178a033a2096bd852e8f43ec3317fb8899f1c2ad254595514e00b3055bd3b622770eb29fda9a45299d921f1d4ad7b'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.models import Sequential
import matplotlib.pyplot as plt
from keras.utils import plot_model


In [ ]:
model = Sequential([
  tf.keras.layers.Conv2D(256, (3, 3), activation='relu',input_shape=(240, 240, 3)),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(8, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Implement data augmentation here (using libraries like Keras ImageDataGenerator)
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        rotation_range=30,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    "/kaggle/input/indian-currency-note-images-dataset-2020/Indian currency dataset v1/training",
    target_size=(240,240),
    batch_size=32,
    shuffle=True,
    seed=42,
    color_mode="rgb",
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    "/kaggle/input/indian-currency-note-images-dataset-2020/Indian currency dataset v1/validation",
    target_size=(240,240),
    batch_size=32,
    shuffle=True,
    seed=42,
    color_mode="rgb",
    class_mode='categorical')
model.summary()

Found 3566 images belonging to 8 classes.
Found 345 images belonging to 8 classes.


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_28 (Conv2D)              │ (None, 238, 238, 256)  │         7,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_28 (MaxPooling2D) │ (None, 119, 119, 256)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 117, 117, 128)  │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_29 (MaxPooling2D) │ (None, 58, 58, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_30 (Conv2D)              │ (None, 56, 56, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_30 (MaxPooling2D) │ (None, 28, 28, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_31 (Conv2D)              │ (None, 26, 26, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_31 (MaxPooling2D) │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_32 (Conv2D)              │ (None, 11, 11, 16)     │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_32 (MaxPooling2D) │ (None, 5, 5, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 512)            │       205,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 8)              │         1,032 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 769,656 (2.94 MB)

 Trainable params: 769,656 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
epochs = 75
history = model.fit(x = train_generator,validation_data=test_generator ,batch_size=32,verbose=1, epochs=epochs)

Epoch 1/75


W0000 00:00:1711271271.492084      90 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1339 - loss: 2.1061

W0000 00:00:1711271432.595466      88 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


112/112 ━━━━━━━━━━━━━━━━━━━━ 235s 2s/step - accuracy: 0.1339 - loss: 2.1060 - val_accuracy: 0.1304 - val_loss: 2.0737
Epoch 2/75
112/112 ━━━━━━━━━━━━━━━━━━━━ 154s 1s/step - accuracy: 0.1566 - loss: 2.0941 - val_accuracy: 0.1333 - val_loss: 2.1648
Epoch 3/75
112/112 ━━━━━━━━━━━━━━━━━━━━ 154s 1s/step - accuracy: 0.2308 - loss: 1.9418 - val_accuracy: 0.3884 - val_loss: 1.7602
Epoch 4/75
112/112 ━━━━━━━━━━━━━━━━━━━━ 158s 1s/step - accuracy: 0.3290 - loss: 1.7778 - val_accuracy: 0.3768 - val_loss: 1.6185
Epoch 5/75
112/112 ━━━━━━━━━━━━━━━━━━━━ 157s 1s/step - accuracy: 0.3902 - loss: 1.6780 - val_accuracy: 0.3246 - val_loss: 1.6619
Epoch 6/75
112/112 ━━━━━━━━━━━━━━━━━━━━ 156s 1s/step - accuracy: 0.3950 - loss: 1.6132 - val_accuracy: 0.4319 - val_loss: 1.4831
Epoch 7/75
112/112 ━━━━━━━━━━━━━━━━━━━━ 160s 1s/step - accuracy: 0.4506 - loss: 1.4912 - val_accuracy: 0.4754 - val_loss: 1.3166
Epoch 8/75
112/112 ━━━━━━━━━━━━━━━━━━━━ 160s 1s/step - accuracy: 0.4807 - loss: 1.4523 - val_accuracy: 0.518

In [ ]:
model.evaluate(test_generator)

11/11 ━━━━━━━━━━━━━━━━━━━━ 10s 899ms/step - accuracy: 0.8686 - loss: 0.6023


[0.6256293654441833, 0.8927536010742188]

In [ ]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array  # Replace with your image loading library if using different

# Define the path to your folder containing images
image_folder = "/kaggle/input/indian-currency-note-images-dataset-2020/Indian currency dataset v1/test"

# Load your trained model
model = tf.keras.models.load_model("/kaggle/working/updatedmodel.h5")  # Replace tf.keras with your model loading method

# Define the target image size expected by your model (height, width)
img_height, img_width = (240, 240)  # Update these values as per your model

# List to store predictions
predictions = []

# Loop through each image in the folder
for filename in os.listdir(image_folder):
    # Check if it's an image file
    if filename.endswith(".jpg") or filename.endswith(".png"):  # Add other extensions if needed

        # Load the image
        img = load_img(os.path.join(image_folder, filename), target_size=(img_height, img_width))

        # Preprocess the image for prediction (convert to array, normalize etc.)
        img_array = img_to_array(img)
        img_array = img_array / 255.0  # Assuming normalization by 255, adjust if needed
        img_array = np.expand_dims(img_array, axis=0)  # Add extra dimension for batch prediction

        # Make prediction using the model
        prediction = model.predict(img_array)

        # Process the prediction (get class label, probability etc.)
        # This part depends on your model's output format
        predicted_class = np.argmax(prediction[0])  # Assuming classification model
        predicted_prob = prediction[0][predicted_class] * 100  # Assuming probability in output

        # Store prediction results (filename, class, probability etc.)
        predictions.append((filename, predicted_class, predicted_prob))

# Print or use the predictions as needed
for filename, pred_class, pred_prob in predictions:
    print(f"Image: {filename}, Predicted Class: {pred_class}, Probability: {pred_prob:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━

In [ ]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array  # Replace with your image loading library if using different

# Define the path to your folder containing images
image_folder = "/kaggle/input/indian-currency-note-images-dataset-2020/Indian currency dataset v1/test"

# Load your trained model
model = tf.keras.models.load_model("/kaggle/working/updatedmodel.h5")  # Replace tf.keras with your model loading method

# Define the target image size expected by your model (height, width)
img_height, img_width = (240, 240)  # Update these values as per your model

# Extract the denomination information from filenames (assuming format: denomination[_.]*_*.jpg)
def get_denomination(filename):
  parts = filename.split("_")
  # Check if denomination is the first part or before the first underscore
  if len(parts) > 1 and parts[0].isdigit():
    return int(parts[0])
  else:
    return None  # Return None if denomination not found in expected format

# List to store predictions
predictions = []

# Loop through each image in the folder
for filename in os.listdir(image_folder):
    # Check if it's an image file
    if filename.endswith(".jpg") or filename.endswith(".png"):  # Add other extensions if needed

        # Load the image
        img = load_img(os.path.join(image_folder, filename), target_size=(img_height, img_width))

        # Preprocess the image for prediction (convert to array, normalize etc.)
        img_array = img_to_array(img)
        img_array = img_array / 255.0  # Assuming normalization by 255, adjust if needed
        img_array = np.expand_dims(img_array, axis=0)  # Add extra dimension for batch prediction

        # Make prediction using the model
        prediction = model.predict(img_array)

        # Process the prediction (get class label, probability etc.)
        # This part depends on your model's output format
        predicted_class = np.argmax(prediction[0])  # Assuming classification model
        predicted_prob = prediction[0][predicted_class] * 100  # Assuming probability in output

        # Extract denomination from filename
        denomination = get_denomination(filename)

        # Combine prediction with filename information
        predicted_text = f"Predicted Class: {predicted_class}, Probability: {predicted_prob:.2f}%"
        if denomination is not None and denomination != predicted_class:  # Check if denomination found and mismatch
            predicted_text += f" (Filename Denomination: {denomination})"  # Add filename denomination if mismatch

        # Store prediction results (filename, predicted class, probability etc.)
        predictions.append((filename, predicted_text))

# Print or use the predictions as needed
for filename, prediction_text in predictions:
    print(f"Image: {filename}, {prediction_text}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 402ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━

In [ ]:
loss_train = history.history['loss']
loss_val = history.history['val_loss']
ep = range(epochs)
plt.plot(ep, loss_train, label='Training loss')
plt.plot(ep, loss_val, label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
loss_train = history.history['accuracy']
loss_val = history.history['val_accuracy']
ep = range(epochs)
plt.plot(ep, loss_train,label='Training accuracy')
plt.plot(ep, loss_val, label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
model.save('updatedmodel.h5')